In [43]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data")
docs = loader.load()
docs

[Document(page_content="Course Title Course Link Description Course length Learner Level\nGit & GitHub - The Practical Guide https://www.udemy.com/course/git-github-practical-guide/?couponCode=LETSLEARNNOWP Learn Git & GitHub and master working with commits, branches, the stash, cherry picking, rebasing, pull requests & more 10.5 hours All Level\nGit & GitHub for Beginners & Integration With Popular IDEs https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP Practical Approach to Version Control Git and GitHub. Learn How Git Works With IntelliJ, Pycharm, Visual Studio, Eclipse 4 hours All Level\nGIT: Advanced commands https://www.udemy.com/course/git-advanced-commands/?couponCode=LETSLEARNNOWPP Practical GIT training: cherry pick, rebase, revert, stash, clean, and more. Take your GIT skills to the next level 4 hours All Level\nGit & GitHub - The Complete Git & GitHub Course https://www.udemy.com/course/git-github-the-complete-git-

In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
documents = text_splitter.split_documents(docs)

In [45]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [46]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [47]:
db

In [48]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser



In [49]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [50]:
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [51]:
llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")

In [52]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}. also provide Course link, duration and level of the course.

Prepare 3 sets of courses based on the total duration of the courses and each set should have 
at least 2 course.

The describution for the sets are following:
    - Set 1 Which we called "1 day plan", total duration of all the courses in set 1 should be less then 5 hours
    - Set 2 Which we called "3 day plan", total duration of all the courses in set 2 should be grater then 12 hours but less then 15 hours
    - Set 3 Which we called "5 day plan", total duration of all the courses in set 3 should be grater then 22 hours but less then 25 hours

Do not suggest courses which you have already suggested.

You must provide the following things in the answer:
    - Course Name
    - Course Link
    - Course Duration
    - Course Level



Be as concise as possible when giving answers. Do not anounce that you will answering. 
Do not say "Let's get started", or "Based on the skills you provided, I would recommend the 
following courses to help you enhance your knowledge and skills : " or "Here are some courses"
and do not introduce your answer. Just write the your answer.

<context>
{context}
<context>

skills : {skills}

experience : {experience}

"""
)

In [53]:
retriever = db.as_retriever()

In [54]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

In [55]:
chain = setup | prompt | llm | parser

In [56]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})

In [57]:
print(response)

Here are the recommended courses for each set:

**Set 1: 1-day plan**

* **Python for Everybody (3 hours)**: https://www.coursera.org/specializations/python-for-everybody
	+ Level: Beginner
	+ Duration: 3 hours
	+ Description: Learn the basics of Python programming
* **Introduction to Artificial Intelligence (2 hours)**: https://www.coursera.org/learn/introduction-artificial-intelligence
	+ Level: Beginner
	+ Duration: 2 hours
	+ Description: Learn the basics of Artificial Intelligence

**Set 2: 3-day plan**

* **Python Machine Learning (6 hours)**: https://www.coursera.org/specializations/python-machine-learning
	+ Level: Intermediate
	+ Duration: 6 hours
	+ Description: Learn the basics of Machine Learning with Python
* **Deep Learning with Python (6 hours)**: https://www.coursera.org/specializations/deep-learning-python
	+ Level: Intermediate
	+ Duration: 6 hours
	+ Description: Learn the basics of Deep Learning with Python

**Set 3: 5-day plan**

* **Python Data Structures (8 hours

In [58]:
response_2 = chain.invoke({"skills": "Java, AWS and Docker",
              "experience": 1})

print(response_2)

Here are three sets of courses based on the given context:

**Set 1: 1 day plan (total duration: 4.5 hours)**

* Course 1: "The Docker for DevOps Course from Development to Production" (8 hours, All Levels, https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/)
* Course 2: "DevOps Tools and AWS for Java Microservice Developers" (20.5 hours, All Levels, https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/)

**Set 2: 3 day plan (total duration: 14.5 hours)**

* Course 1: "The Docker for DevOps Course from Development to Production" (8 hours, All Levels, https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/)
* Course 2: "DevOps Tools and AWS for Java Microservice Developers" (20.5 hours, All Levels, https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/)

**Set 3: 5 day plan (total duration: 24 hours)**

* Course 1: "The Docker for DevOps Course from Developm

In [59]:
response_3 = chain.invoke({"skills": "DevOps and AWS",
              "experience": 2})

print(response_3)

Here are the suggested courses for each set:

**Set 1: 1 Day Plan**

1. The Docker for DevOps Course from Development to Production (8 hours, All Levels) - https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
2. DevOps Tools and AWS for Java Microservice Developers (20.5 hours, All Levels) - https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/

**Set 2: 3 Day Plan**

1. The Complete Cloud Computing Software Architecture Patterns (4.5 hours, All Levels) - https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
2. Design and Develop a Killer Website with HTML5 and CSS3 (37.5 hours, All Levels) - https://www.udemy.com/course/design-and-develop-a-killer-website-with-html5-and-css3/

**Set 3: 5 Day Plan**

1. DevOps: Jenkins CI/CD for Kubernetes Deployments (20.5 hours, All Levels) - https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
2. HTML and CSS for Begi

In [60]:
response_4 = chain.invoke({"skills": "DevOps",
              "experience": 2})

print(response_4)

Set 1: "1 day plan" (Total duration: 3.5 hours)

* Course Name: The Docker for DevOps Course from Development to Production
	+ Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	+ Course Duration: 8 hours
	+ Course Level: All Levels
* Course Name: DevOps Tools and AWS for Java Microservice Developers
	+ Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	+ Course Duration: 20.5 hours
	+ Course Level: All Levels

Set 2: "3 day plan" (Total duration: 14 hours)

* Course Name: Master Docker, Kubernetes, Maven, Jenkins, GIT, AWS EC2, Elastic Beanstalk, ELB, Auto Scaling and more in easy steps
	+ Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	+ Course Duration: 20.5 hours
	+ Course Level: All Levels
* Course Name: HTML and CSS for Beginners - Build a Website & Launch ONLINE
	+ Course Link: https://www.udemy.com/course/design-and-develop-a-kill

### EX 2:

In [67]:
prompt_2=ChatPromptTemplate.from_template(
"""
Suggest the best courses for Talent Transformation based on the provided {skills} and {experience}.


Prepare 3 sets of courses based on total learning hours:
- Set 1 (1-day plan): only suggest courses whose Course length < 3 hours
- Set 2 (3-day plan): only suggest courses whose Course length is grater then 3 hours and less then 20 hours
- Set 3 (5-day plan): only suggest courses whose Course length is grater then 20 hours

Also include at least 2 courses for Git & GitHub, Linex and SAP BTP.
Each set should include at least 2 courses. Avoid suggesting the same courses.

Do not suggest courses from outside. strickly follow the context



Requirements:
- Course Name
- Course Link
- Course Duration
- Course Level


Be as concise as possible when giving answers. Do not anounce that you will answering. 
Do not say "Let's get started", or "Based on the skills you provided, I would recommend the 
following courses to help you enhance your knowledge and skills : " or "Here are some courses"
and do not introduce your answer. Just write the your answer.

<context>
{context}
<context>

Skills: {skills}

Experience: {experience}
"""
)


In [68]:
chain_2 = setup | prompt_2 | llm | parser

In [69]:
response = chain_2.invoke({"skills": "python and AI",
              "experience": 2.5})

print(response)

Set 1 (1-day plan):
1. Statistics for Data Science and Business Analysis
   - Course Link: https://www.udemy.com/course/statistics-for-data-science-and-business-analysis/
   - Course Duration: 5 hours
   - Course Level: All Levels

2. AWS Business Essentials - The Business Value of AWS
   - Course Link: https://www.udemy.com/course/aws-business-essentials/
   - Course Duration: 2.5 hours
   - Course Level: Beginner

Set 2 (3-day plan):
1. Design and Develop a Killer Website with HTML5 and CSS3
   - Course Link: https://www.udemy.com/course/design-and-develop-a-killer-website-with-html5-and-css3/
   - Course Duration: 37.5 hours
   - Course Level: All Levels

2. DevOps Tools and AWS for Java Microservice Developers
   - Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
   - Course Duration: 20.5 hours
   - Course Level: All Levels

Set 3 (5-day plan):
1. Data Science A-Z: Hands-On Exercises & ChatGPT Prize [2024]
   - Course Link: https://w

In [70]:
response_2 = chain_2.invoke({"skills": "Java, AWS and Docker",
              "experience": 1})

print(response_2)

**Set 1 (1-day plan)**
1. DevOps Tools and AWS for Java Microservice Developers (2 hours, All Levels)
	* https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
2. The Docker for Devops Course: From Development to Production (8 hours, All Levels)
	* https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/

**Set 2 (3-day plan)**
1. The Complete Cloud Computing Software Architecture Patterns (4.5 hours, All Levels)
	* https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
2. Devops Tools and AWS for Java Microservice Developers (20.5 hours, All Levels)
	* https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/

**Set 3 (5-day plan)**
1. Devops Tools and AWS for Java Microservice Developers (20.5 hours, All Levels)
	* https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
2. HTML and CSS for Beginners - Build a Website & Launch ONLINE (3

In [71]:
response_3 = chain_2.invoke({"skills": "DevOps and AWS",
              "experience": 2})

print(response_3)

Set 1 (1-day plan):

1. The Docker for DevOps Course from Development to Production - 8 hours
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Duration: 8 hours
	* Course Level: All Levels
2. DevOps Tools and AWS for Java Microservice Developers - 20.5 hours
	* Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	* Course Duration: 20.5 hours
	* Course Level: All Levels

Set 2 (3-day plan):

1. The Complete Cloud Computing Software Architecture Patterns - 4.5 hours
	* Course Link: https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
	* Course Duration: 4.5 hours
	* Course Level: All Levels
2. Design and Develop a Killer Website with HTML5 and CSS3 - 37.5 hours
	* Course Link: https://www.udemy.com/course/design-and-develop-a-killer-website-with-html5-and-css3/
	* Course Duration: 37.5 hours
	* Course Level: All Levels

Set 3 (5-day plan):


In [72]:
response_4 = chain_2.invoke({"skills": "DevOps",
              "experience": 2})

print(response_4)

Set 1 (1-day plan):
1. 'DevOps Tools and AWS for Java Microservice Developers' (2 hours)
	* https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	* 20.5 total hours
	* All Levels
2. 'The Docker for DevOps Course from Development to Production' (1 hour)
	* https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* 8 total hours
	* All Levels

Set 2 (3-day plan):
1. 'The Complete Cloud Computing Software Architecture Patterns' (4.5 hours)
	* https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
	* 4.5 total hours
	* All Levels
2. 'Design and Develop a Killer Website with HTML5 and CSS3' (37.5 hours)
	* https://www.udemy.com/course/design-and-develop-a-killer-website-with-html5-and-css3/
	* 37.5 total hours
	* All Levels

Set 3 (5-day plan):
1. 'HTML and CSS for Beginners - Build a Website & Launch ONLINE' (37.5 hours)
	* https://www.udemy.com/course/html-and-css-for-beginners-build-a-webs